In [12]:
import json
dataset=json.load(open('conversation.json'))

conv=dataset['conversations']

data=[]
target=[]

In [13]:
for i in range(len(conv)):
    for j in range(len(conv[i])):
        if j<len(conv[i])-1:
            data.append(conv[i][j])
            target.append(conv[i][j+1])

In [15]:
import nltk
nltk.download('punkt')

data_tok=[nltk.word_tokenize(dialog) for dialog in data]
target_tok=[nltk.word_tokenize(dialog) for dialog in target]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MasTeR\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from gensim import models

model=models.Word2Vec.load('word2vec.bin')

data_vec=[]
target_vec=[]

for sentence in data_tok:
    print(sentence)

C:\Users\MasTeR\Anaconda3\lib\site-packages\smart_open\ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')
C:\Users\MasTeR\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [19]:
from gensim.models import Word2Vec,KeyedVectors

model=KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin',binary=True,limit=100000)


In [21]:
data_vec=[]
target_vec=[]
for sentence in data_tok:
    sentence_vector=[model[word] for word in sentence if word in model.vocab]
    data_vec.append(sentence_vector)
    
for sentence in target_tok:
    sentence_vector=[model[word] for word in sentence if word in model.vocab]
    target_vec.append(sentence_vector)

In [23]:
import numpy as np

sentence_end=np.ones((300,),dtype=np.float32)

for vector in data_vec:
    vector[14:]=[] #limitting the words in a sentence to 15
    vector.append(sentence_end)

for vector in data_vec:
    if(len(vector)<15):
        for i in range(15-len(vector)):
            vector.append(sentence_end) #filling the empty words in a sentence


for vector in target_vec:
    vector[14:]=[] #limitting the words in a sentence to 15
    vector.append(sentence_end)

for vector in target_vec:
    if(len(vector)<15):
        for i in range(15-len(vector)):
            vector.append(sentence_end) #filling the empty words in a sentence

In [3]:
#implementing neural network

from keras.models import Sequential
from keras.layers.recurrent import LSTM,SimpleRNN #LSTM neuron type
from sklearn.model_selection import train_test_split

data_vec=np.array(data_vec)
target_vec=np.array(target_vec)

train_data,test_data,train_target,test_target=train_test_split(data_vec,target_vec,test_size=0.1)

Using TensorFlow backend.


NameError: name 'np' is not defined

In [ ]:
#neuarl netowrk model

model=Sequential()
model.add(LSTM(output_dim=300,input_shape=(15,300),return_sequences=True,init='glorot_normal',inner_init='glorot_normal',activation='sigmoid'))
model.add(LSTM(output_dim=300,input_shape=(15,300),return_sequences=True,init='glorot_normal',inner_init='glorot_normal',activation='sigmoid'))
model.add(LSTM(output_dim=300,input_shape=(15,300),return_sequences=True,init='glorot_normal',inner_init='glorot_normal',activation='sigmoid'))
model.add(LSTM(output_dim=300,input_shape=(15,300),return_sequences=True,init='glorot_normal',inner_init='glorot_normal',activation='sigmoid'))

model.compile(loss='cosine_proximity',optimizer='adam',metrics=['accuracy'])

#model.fit(train_data,train_target,epochs=500)

In [1]:
model.load_weights('LSTM5000.h5') #we get pre train weight an byers h5 file

NameError: name 'model' is not defined

In [ ]:
from gensim.models import Word2Vec,KeyedVectors

model_w2v=KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin',binary=True,limit=100000)


In [ ]:
#creating front end
import nltk
import numpy as np

sentence_end=np.ones((300),dtype=np.float32)
while(True):
    msg=input('Enter the msg:')
    sentence_tok=nltk.word_tokenize(msg.lower()) #lower for alll simple letters
    sentence_vec=[model_w2v[word] for word in sentence_tok if word in model_w2v.vocab]
    
    sentence_vec[14:]=[]
    sentence_vec.append(sentence_end)
    
    if len(sentence_vec)<15:
        for i in range(15-len(sentence_vec)):
            sentence_vec.append(sentence_end)
    
    sentence_vec=np.array([sentence_vec])
    
    result=model.predict(sentence_vec)
    out_msg=[model_w2v.most_similar([results[0][i]])[0][0] for i in range(15)]
    out_msg=''.json(out_msg)
    print(out_msg)